<a href="https://colab.research.google.com/github/Onkar-stac/Next-Word-Predictor/blob/main/nextwordpredictor_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import library
import re
import requests
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
# data to extract
url = "https://www.gutenberg.org/cache/epub/5200/pg5200.txt"

In [ ]:
#function to extract data from the url
def get_book(url):
  raw = requests.get(url).text
  #discarding the beginning of the data
  start = re.search(r"\*\*\* START OF THIS PROJECT GUTENBERG EBOOK .* \*\*\*",raw).end()
  #discarding the end of the data
  stop=re.search(r"II", raw).start()
  text=raw[start:stop] #relevant data
  return text

In [ ]:
#processing
def preprocess(sentence):
  return re.sub('[^A-Za-z0-9.]+',' ',sentence).lower()

In [ ]:
#calling the functions
book = get_book(url)
processed_book=preprocess(book)
print(processed_book)

In [ ]:
len(processed_book)

In [ ]:
# EDA
len(re.findall(r'the',processed_book))
processed_book = re.sub(r'\si\s', " I ",processed_book)
processed_book = re.sub(r'[^\w\s]'," ",processed_book)
print(processed_book)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([processed_book])
#saving the tokenizer
pickle.dump(tokenizer,open('token.pkl','wb'))
sequence_data = tokenizer.texts_to_sequences([processed_book])[0] #vectorize the text corpus
sequence_data[:15]

In [ ]:
len(sequence_data)

In [ ]:
vocab_size = len(tokenizer.word_index)+1 #number of unique words in the text corpus
print(vocab_size)

In [ ]:
sequences = []
for i in range(3,len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)
print("The length of sequences are: ",len(sequences))
sequences = np.array(sequences)
sequences[:10]

In [ ]:
X=[]
Y=[]
for i in sequences:
  X.append(i[0:3])
  Y.append(i[3])
X=np.array(X)
Y=np.array(Y)


In [ ]:
Y=to_categorical(Y,num_classes=vocab_size)
Y[:5]

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size,10,input_length=3))
model.add(LSTM(1000))
model.add(Dropout(0.2))
model.add(Dense(vocab_size,activation="softmax"))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
file_name_path="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(file_name_path, monitor='loss', 
verbose=1, save_best_only=True, mode='min')
model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.001))
callbacks = [checkpoint]

In [ ]:
model.fit(X, Y, epochs=50, batch_size=64, callbacks=callbacks) 

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle
model = load_model('weights-improvement-50-0.6095.hdf5')
tokenizer = pickle.load(open('token.pkl','rb'))

def Predict_Next_Words(model,tokenizer,text):
  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key,value in tokenizer.word_index.items():
    if value==preds:
      predicted_word=key
      break
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter: ")
  if text=="0":
    print("Execution completed")
    break
  else:
    text=text.split(" ")
    text=text[-3:]
    print(text)
    Predict_Next_Words(model,tokenizer,text)